In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 라이브러리 로드 및 초기설정

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from datetime import datetime, timedelta

from tqdm import tqdm

In [3]:
# 하나의 cell에서 print문을 사용하지 않고 multiple output 출력을 가능하게 하는 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# DataFrame의 사이즈가 큰 경우, 어떻게 화면에 출력을 할지 setting하는 코드
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('max_columns', None)

In [4]:
# 시스템 운영체제에 맞는 폰트를 불러와 적용하기
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

font_family = get_font_family()

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [5]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당합니다.

# 폰트설정
plt.rc("font", family=font_family)

# 마이너스 폰트 설정
plt.rc("axes", unicode_minus=False)

# 그래프에 retina display 적용
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

## 데이터 저장 폴더

In [6]:
path_dir = '/content/gdrive/MyDrive/aiffel-hackathon/data'
file_list = os.listdir(path_dir)

In [7]:
file_list

['상가업소정보_201912_01.csv',
 '상가업소정보_201912_02.csv',
 '상가업소정보_201912_03.csv',
 '상가업소정보_201912_04.csv',
 '20211022_sample_hackathon_data.csv',
 '교문1동_일최고기온_201901_201911.csv',
 '교문1동_하늘상태_201901_201911.csv',
 '교문1동_3시간기온_201901_201911.csv',
 '교문1동_일최저기온_201901_201911.csv',
 '교문1동_강수형태_201901_201911.csv',
 '교문1동_6시간적설_201901_201911.csv',
 '교문1동_6시간강수량_201901_201911.csv',
 '교문1동_풍속_201901_201911.csv',
 '교문1동_습도_201901_201911.csv',
 '교문1동_파고_201901_201911.csv',
 '교문1동_풍향_201901_201911.csv',
 '의정부1동_강수형태_201901_201911.csv',
 '의정부1동_강수확률_201901_201911.csv',
 '의정부1동_일최고기온_201901_201911.csv',
 '의정부1동_6시간적설_201901_201911.csv',
 '의정부1동_일최저기온_201901_201911.csv',
 '의정부1동_3시간기온_201901_201911.csv',
 '의정부1동_하늘상태_201901_201911.csv',
 '의정부1동_6시간강수량_201901_2

## 데이터 전처리

In [8]:
# 지역별 습도 데이터 전처리
def preprocess_hum(df, region_name):

    df.columns = ["day", "hour", "forecast", "hum"]
    x = 1

    for idx, row in tqdm(df.iterrows()):

        df.loc[idx, 'day']= '2019-' + str(x) + '-' + row['day']
        if 'Start' in row['day']:
            #print(row['day'])
            df = df.drop([idx])
            x += 1
            #print('2019-0' + str(x) + '-' + row['day'])
    
    df['day'] = df['day'].str.replace(" ", "")
    df['hour'] = df['hour'].astype(int).astype(str)
    df['hour'] = df['hour'].apply(lambda x: x[:-2] + ":" + x[-2:])

    df['pred_date'] = 0

    for idx, row in tqdm(df.iterrows()):
        df.loc[idx, 'pred_date'] = pd.to_datetime(row['day'] + "-" + row['hour']) - timedelta(hours=row['forecast'])

    df['pred_date'] = pd.to_datetime(df['pred_date'])
    df['day'] = pd.to_datetime(df['day'])

    df['d-day'] = (df['day'] - df['pred_date']).dt.days
    df['d-day'] = df['d-day'] + 1

    pivot_df = pd.pivot_table(df, index=['day'], columns='d-day', values='hum', aggfunc='mean')
    
    ## 서윤님꺼에 합치기
    pivot_df = pivot_df.reset_index()
    pivot_df['region'] = region_name
    pivot_df.columns = ['day', 'd_0', 'd_1', 'd_2', 'd_3', 'region']
    return df, pivot_df

In [10]:
gimpo = pd.read_csv(path_dir + '/사우동_습도_201901_201911.csv')  # 경기 김포시
df_gimpo, df_pivot_gimpo = preprocess_hum(gimpo, '경기 김포시')

uijeongbu = pd.read_csv(path_dir + '/의정부1동_습도_201901_201911.csv')  # 경기 의정부시
df_uijeongbu, df_pivot_uijeongbu = preprocess_hum(uijeongbu, '경기 의정부시')

namyangju = pd.read_csv(path_dir + '/금곡동_습도_201901_201911.csv')  # 경기 남양주시
df_namyangju, df_pivot_namyangju = preprocess_hum(namyangju, '경기 남양주시')

guri = pd.read_csv(path_dir + '/교문1동_습도_201901_201911.csv')  # 경기 구리시
df_guri, df_pivot_guri = preprocess_hum(guri, '경기 구리시')

hanam = pd.read_csv(path_dir + '/신장1동_습도_201901_201911.csv')  # 경기 하남시
df_hanam, df_pivot_hanam = preprocess_hum(hanam, '경기 하남시')

gwangmyeong = pd.read_csv(path_dir + '/철산1동_습도_201901_201911.csv')  # 경기 광명시
df_gwangmyeong, df_pivot_gwangmyeong = preprocess_hum(gwangmyeong, '경기 광명시')

49442it [01:11, 695.30it/s]
49432it [01:53, 434.95it/s]
49442it [01:14, 663.50it/s]
49432it [02:01, 406.96it/s]
49442it [01:15, 654.33it/s]
49432it [01:56, 425.33it/s]
49442it [01:16, 648.04it/s]
49432it [02:00, 409.64it/s]
49442it [01:15, 655.97it/s]
49432it [01:57, 420.28it/s]
49442it [01:17, 641.40it/s]
49432it [01:52, 437.65it/s]


In [11]:
# 지역별 데이터 병합
kyunghyun_humid = pd.concat([df_pivot_gimpo, df_pivot_uijeongbu, df_pivot_namyangju,
                            df_pivot_guri, df_pivot_hanam, df_pivot_gwangmyeong])

# 데이터 저장
kyunghyun_humid.to_csv('/content/gdrive/MyDrive/aiffel-hackathon/data/preprocess/kyunghyun_humid.csv', index=False)

### 데이터 확인

In [12]:
# 습도
df_humid = pd.read_csv('/content/gdrive/MyDrive/aiffel-hackathon/data/preprocess/kyunghyun_humid.csv')
df_humid

,day,d_0,d_1,d_2,d_3,region
0,2019-01-01,54.286,55.000,57.900,62.500,경기 김포시
1,2019-01-02,56.071,58.203,58.000,67.500,경기 김포시
2,2019-01-03,64.821,58.281,50.000,42.500,경기 김포시
3,2019-01-04,53.750,52.500,52.500,57.500,경기 김포시
4,2019-01-05,53.036,56.797,57.900,63.333,경기 김포시
...,...,...,...,...,...,...
1999,2019-11-26,54.643,60.000,60.700,63.333,경기 광명시
2000,2019-11-27,60.536,59.766,60.200,52.500,경기 광명시
2001,2019-11-28,55.714,59.062,64.700,62.500,경기 광명시
2002,2019-11-29,61.250,68.750,69.000,77.500,경기 광명시
